In [5]:
import numpy  as np

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.palettes import Inferno, all_palettes
from bokeh.models import CustomJS, Slider, ColumnDataSource 
from bokeh.models import Whisker, HoverTool, Span, ColorBar
from bokeh.transform import linear_cmap, log_cmap, factor_cmap

output_notebook()

Loading BokehJS ...

In [6]:
def plot_histogram(array):
    plot_options = dict(width=450,
                        plot_height=250,
                        tools='pan,wheel_zoom,reset,save')

    gate_hist = figure(**plot_options)

    hist, edges = np.histogram(array, bins=50)

    x = np.linspace(0, 1, 1000)

    gate_hist.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="#036564", line_color="#033649",\
    )

    
    
    gate_hist.xaxis.axis_label = 'X'
    gate_hist.yaxis.axis_label = 'Density'
    show(gate_hist)

In [26]:
d = 3
n_samples = 1000

cos_sims = []

for i in range(n_samples):
    ## sampling from a hypercube
#     vec1 = np.random.uniform(-1, 1, size=(d))
#     vec2 = np.random.uniform(-1, 1, size=(d))
    
    ## sampling from a hypersphere
    u = np.random.normal(0,1,d)  # an array of d normally distributed random variables
    norm=np.sum(u**2) **(0.5)
    r = np.random.random()**(1.0/d)
    vec1 = r*u/norm
    
    u = np.random.normal(0,1,d)  # an array of d normally distributed random variables
    norm=np.sum(u**2) **(0.5)
    r = np.random.random()**(1.0/d)
    vec2 = r*u/norm
    
    cos_sims.append(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))

plot_histogram(cos_sims)